CPace sage notebook for generating test vectors for the CPace cipher suites.

The notebook is organized in a series of several subcells.
1.) X25519 definitions and basic string->integer->fieldElement->string conversions back and forth.
2.) Definitions of the Elligator2 primitive straight-line code from the Elligator paper, checked also against the code from the hash_to_curve team.
3.) Test vector generation for X25519 and Elligator2 as ported to sage from the ANSI-C code from the
    Endress+Hauser crypto library.
4.) Implementation and test vector generation for the CPace-specific constructions for generator
    calculation and ISK determination.
5.) Implementation and test vector generation for the AuCPace-specific constructions for generator
    calculation and SK determination.
    
The cells should best be calculated in sequence so that all definitions are available.
Tested with SageMath 9.0 on ubuntu linux.

In [69]:
# 1.) X25519 definitions and tool definitions
   
def ByteArrayToInteger(k,numBytes=32):
    return sum((k[i] << (8 * i)) for i in range(len(k)))

def IntegerToByteArray(k,numBytes = 32):
    result = bytearray(numBytes);
    for i in range(numBytes):
        result[i] = (k >> (8 * i)) & 0xff;
    return result

def IntegerToLEPrintString(u,numBytes=32):
    u = Integer(u)
    res = ""
    ctr = 0
    while ((u != 0) | (numBytes > 0)):
        byte =  u % 256
        res += ("%02x" % byte)
        u = (u - byte) >> 8
        numBytes = numBytes - 1
        ctr = ctr + 1
    return res

def ByteArrayToCInitializer(k, name):
    values = [b for b in k]
    result = "const uint8_t " + name +"[] = {\n "
    for x in values:
        result += hex(x) +","
    result += "\n};"
    return result

def ByteArrayToLEPrintString(k):
    bytes = [(b) for b in k]
    res = ""
    ctr = 0
    for x in bytes:
        res += ("%02x" %x)
        ctr = ctr + 1
    return res

def tv_output_byte_array(data, test_vector_name = "", line_prefix = "  ", max_len = 70):
    string = ByteArrayToLEPrintString(data)
    print (test_vector_name + ":",end="")
    chars_per_line = max_len - len(line_prefix)
    while True:
        print ("\n" + line_prefix + string[0:chars_per_line],end="")
        string = string[chars_per_line:-1]
        if len(string) == 0:
            print("\n",end="")
            return

In [13]:
########## X25519 ##################

def clampScalar25519(k):
    r = bytearray(k)
    r[0] &= 248
    r[31] &= 127
    r[31] |= 64
    return r

def clampScalarForInversion25519(k):
    r = bytearray(k)
    r[0] &= 248
    return r

# all inputs to be given as byte array.
def Inverse_X25519(scalar,basepoint):
    OrderPrimeSubgroup = 2^252 + 27742317777372353535851937790883648493
    SF = GF(OrderPrimeSubgroup)
    coFactor = SF(8)
    scalarClamped = clampScalar25519(scalar)
    inverse_scalar = 1 /  (SF(ByteArrayToInteger(scalarClamped)) * coFactor)
    inverse_scalar_int = Integer(inverse_scalar) * 8
    inverse_scalar = IntegerToByteArray(inverse_scalar_int)
    return X25519(basepoint,inverse_scalar,withClamping=0)

def X25519(scalar, basepoint, withClamping=1,warnForPointOnTwist = 1):
    prime = 2^255 - 19
    F = GF(prime)
    A = F(486662)
    nonsquare = F(2)   
    E = EllipticCurve(F, [0, A , 0, 1 , 0])
    Twist = EllipticCurve(F, [0, A * nonsquare, 0, 1 * nonsquare^2, 0])

    u = F(ByteArrayToInteger(basepoint))
    if (withClamping == 1):
        clampedScalar = ByteArrayToInteger(clampScalar25519(scalar))
    else:    
        clampedScalar = ByteArrayToInteger(clampScalarForInversion25519(scalar))

        
    d = 1
    v2 = u^3 + A*u^2 + u
    if not v2.is_square():
        if (warnForPointOnTwist):
            print("Input point is on the twist! "),
        E = Twist
        d = nonsquare
        u = d * u
        v2 = u^3 + A*u^2 * nonsquare + u * nonsquare^2
    v = v2.sqrt()
    
    point = E(u, v)
    (resultPoint_u, resultPoint_v, result_Point_z) = point * clampedScalar
    resultCoordinate = resultPoint_u / d
    
    return IntegerToByteArray(Integer(resultCoordinate))

In [14]:
# 2.) Definitions for the X25519 test cases

class X25519_testCase:
    def __init__(self,u_in, s_in, u_out):
        self.u_in = u_in
        self.s_in = s_in
        self.u_out = u_out

    def runTest(self):
        us = IntegerToByteArray(self.u_in)
        ss = IntegerToByteArray(self.s_in)
        r  = IntegerToByteArray(self.u_out)
        u = X25519(ss,us)
        if (u != r):
            print ("Fail")
            print ("Input u :\n0x%032x\n" % self.u_in)
            print ("Input s :\n0x%032x\n" % self.s_in)
            print ("Correct Result :\n0x%032x\n" % self.u_out)
            print ("Actual Result :\n0x%032x\n" % ByteArrayToInteger(u))
            return False
        print ("Pass")
        return True
    
    def docOutput(self):
        print ("Test case for X25519:")
        print ("u:"),
        print (IntegerToLEPrintString(self.u_in))
        print ("s:"),
        print (IntegerToLEPrintString(self.s_in))
        print ("r:"),
        print (IntegerToLEPrintString(self.u_out))
        

testCases = []

tv = \
    X25519_testCase(0x4c1cabd0a603a9103b35b326ec2466727c5fb124a4c19435db3030586768dbe6,\
                    0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                    0x5285a2775507b454f7711c4903cfec324f088df24dea948e90c6e99d3755dac3)
testCases.append(tv)


tv = X25519_testCase(0x13a415c749d54cfc3e3cc06f10e7db312cae38059d95b7f4d3116878120f21e5,\
                     0xdba18799e16a42cd401eae021641bc1f56a7d959126d25a3c67b4d1d4e9664b,\
                    0x5779ac7a64f7f8e652a19f79685a598bf873b8b45ce4ad7a7d90e87694decb95)
testCases.append(tv)

tv = X25519_testCase(0,\
                     0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                     0)
testCases.append(tv)
    
weakp = []
weakp.append(0)
weakp.append(1)
weakp.append(325606250916557431795983626356110631294008115727848805560023387167927233504) #(which has order 8)
weakp.append(39382357235489614581723060781553021112529911719440698176882885853963445705823) #(which also has order 8)
weakp.append(2^255 - 19 - 1)
weakp.append(2^255 - 19)
weakp.append(2^255 - 19 + 1)
weakp.append(2^255 - 19 + 325606250916557431795983626356110631294008115727848805560023387167927233504)
weakp.append(2^255 - 19 + 39382357235489614581723060781553021112529911719440698176882885853963445705823)
weakp.append(2 * (2^255 - 19) - 1)
weakp.append(2 * (2^255 - 19))
weakp.append(2 * (2^255 - 19) + 1)

s_in = 0xff9a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346af;
for x in weakp:
    tv = X25519_testCase (x,s_in,0)
    testCases.append(tv)


for x in testCases:
    x.runTest()

for x in testCases:
    x.docOutput()


Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Test case for X25519:
u:
e6db6867583030db3594c1a424b15f7c726624ec26b3353b10a903a6d0ab1c4c

s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4

r:
c3da55379de9c6908e94ea4df28d084f32eccf03491c71f754b4075577a28552

Test case for X25519:
u:
e5210f12786811d3f4b7959d0538ae2c31dbe7106fc03c3efc4cd549c715a413

s:
4b66e9d4d1b4673c5ad22691957d6af5c11b6421e0ea01d42ca4169e7918ba0d

r:
95cbde9476e8907d7aade45cb4b873f88b595a68799fa152e6f8f7647aac7957

Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000

s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4

r:
0000000000000000000000000000000000000000000000000000000000000000

Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000

s:
af46e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a22

In [15]:
print ("Points that need to return neutral element when input to ")
print ("plain X25519 that also accept un-normalized inputs with")
print ("bit #255 set in the input point encoding.")

ctr=0;
for x in weakp:
    print ("u"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(x),end="");
    ctr += 1;

    
print ("\nResults for X25519 implementations not clearing bit #255:")
print ("s =", IntegerToLEPrintString(s_in),end="");
print ("rN = X25519(uX, s);")
ctr=0;
for x in weakp:
    r = X25519(IntegerToByteArray(s_in), IntegerToByteArray(x),warnForPointOnTwist=0);
    r = ByteArrayToInteger(r)
    print ("r"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(r),end="");
    ctr += 1;


print ("\nResults for X25519 implementations that clear bit #255:")
print ("s =", IntegerToLEPrintString(s_in),end="");
print ("qN = X25519(uX & ((1 << 255) - 1),s);")
ctr=0;
for x in weakp:
    q = X25519(IntegerToByteArray(s_in), IntegerToByteArray(x & ((1<<255)-1)),warnForPointOnTwist=0);
    q = ByteArrayToInteger(q)
    print ("q"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(q),end="");
    ctr += 1;


Points that need to return neutral element when input to 
plain X25519 that also accept un-normalized inputs with
bit #255 set in the input point encoding.
u0: 0000000000000000000000000000000000000000000000000000000000000000
u1: 0100000000000000000000000000000000000000000000000000000000000000
u2: e0eb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b800
u3: 5f9c95bca3508c24b1d0b1559c83ef5b04445cc4581c8e86d8224eddd09f1157
u4: ecffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u5: edffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u6: eeffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u7: cdeb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b880
u8: 4c9c95bca3508c24b1d0b1559c83ef5b04445cc4581c8e86d8224eddd09f11d7
u9: d9ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
ua: daffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
ub: dbffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

Results for X25

In [79]:
# 4.) Definitions for the CPace string handling primitives

def prepend_length_to_bytes(data):
    length = len(data);
    length_as_utf8_string = chr(length).encode('utf-8') # Will be a single byte in almost all real-world cases.
    return length_as_utf8_string + data 

def prefix_free_cat(*args):
    result = b""
    for arg in args:
        result += prepend_length_to_bytes(arg)
    return result

def zero_bytes(length):
    result = b"\0" * length
    return result

def generator_string(PRS,DSI,CI,sid,s_in_bytes):
    """
    Concat all input fields with prepended length information.
    Add zero padding in the first hash block between PRS and DSI.
    """
    len_zpad = max(0,s_in_bytes - 1 - len(prepend_length_to_bytes(PRS)))
    return (prefix_free_cat(PRS, zero_bytes(len_zpad), DSI, CI, sid), len_zpad)

def random_bytes(length):
    values = [randint(0, 255) for i in range(length)]
    result = b""
    for v in values:
        result += v.to_bytes(1, 'little')
    return result

def oCAT(str1,str2):
    if str2 > str1:
        return str1 + str2
    else:
        return str2 + str1

In [24]:
# Definitions for the hash primitives.
import hashlib 

class H_SHA512:
    def __init__(self):
        self.b_in_bytes = 64
        self.bmax_in_bytes = 64
        self.s_in_bytes = 128
        self.name = "SHA512"
        
    def hash(self,input_str, l = 64):
        m = hashlib.sha512(input_str)
        digest = m.digest()
        return digest[0:l]

class H_SHA256:
    def __init__(self):
        self.b_in_bytes = 32
        self.bmax_in_bytes = 32
        self.s_in_bytes = 64
        self.name = "SHA256"
        
    def hash(self,input_str, l = 32):
        m = hashlib.sha256(input_str)
        digest = m.digest()
        return digest[0:l]

class H_SHAKE256:
    def __init__(self):
        self.b_in_bytes = 64
        self.bmax_in_bytes = 2^128
        self.s_in_bytes = 136
        self.name = "SHAKE256"
        
    def hash(self,input_str, l = 64):
        m = hashlib.shake_256(input_str)
        digest = m.digest(l) # Note: hashlib.shake_256 seems to be buggy in some Sage environments :-(
        return digest


In [71]:
class G_Montgomery:
    """Here we have common definitions for the X448 and X25519"""
    def sample_scalar(self):
        return random_bytes(self.field_size_bytes)

    def decodeLittleEndian(self, b):
        bits = self.field_size_bits
        num_bytes = floor((bits+7)/8)
        return sum([b[i] << 8*i for i in range(num_bytes)])

    def decodeUCoordinate(self, u):        
        u_list = [b for b in u]
        # Ignore any unused bits.
        if self.field_size_bits % 8:
            u_list[-1] &= (1<<(self.field_size_bits%8))-1
        return self.decodeLittleEndian(u_list)

    def encodeUCoordinate(self,u):
        u = u % self.q
        return IntegerToByteArray(u,self.field_size_bytes)
    
    def find_z_ell2(self,F):
        """ Argument: F, a field object, e.g., F = GF(2^255 - 19) """
        ctr = F.gen()
        while True:
            for Z_cand in (F(ctr), F(-ctr)):
                # Z must be a non-square in F.
                if is_square(Z_cand):
                    continue
                return Z_cand
            ctr += 1
        
    def elligator2(self,r):
        q = self.q
        Fq = GF(q)
        A = Fq(self.A)
        B = Fq(1)
    
        # calculate the appropriate non-square as specified in the hash2curve draft.
        u = Fq(self.find_z_ell2(Fq))
        powerForChi = floor((q-1)/2)
    
        v = - A / (1 + u * r^2)
        epsilon = (v^3 + A * v^2 + B * v)^powerForChi
        x = epsilon * v - (1 - epsilon) * A/2
        return self.encodeUCoordinate(Integer(x))

    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        string_hash = H.hash(gen_string, self.field_size_bytes)
        u = self.decodeUCoordinate(string_hash)
        result = self.elligator2(u)
        if print_test_vector_info:
            print ("  ####  Test vectors for calculate_generator with group",self.name," ####")
            print ("  Inputs")
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.")
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", G.DSI)
            print ("    CI =", CI)
            print ("    CI =", ByteArrayToLEPrintString(CI))
            print ("    sid =", ByteArrayToLEPrintString(sid))
            print ("  Outputs")
            tv_output_byte_array(string_hash, test_vector_name = "    hash generator string", 
                                 line_prefix = "      ", max_len = 70)
            tv_output_byte_array(IntegerToByteArray(u), test_vector_name = "    after decoding to coordinate", 
                                 line_prefix = "      ", max_len = 70)
            tv_output_byte_array(result, test_vector_name = "    generator g", 
                                 line_prefix = "      ", max_len = 70)
        return result

    
class G_X25519(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(32)
        self.field_size_bytes = 32
        self.field_size_bits = 255
        self.DSI = b"CPace255"
        self.DSI_ISK = b"CPace255_ISK"
        self.name = "X25519" # group name
        
        # curve definitions
        self.q = 2^255 - 19
        self.A = 486662
        
    def scalar_mult(self,scalar,point):
        return X25519(scalar,point)

    def scalar_mult_vfy(self,scalar,point):
        return X25519(scalar,point)

    
class G_X448(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(56)
        self.field_size_bytes = 56
        self.field_size_bits = 448
        self.DSI = b"CPace448"
        self.DSI_ISK = b"CPace448_ISK"
        self.name = "X448" # group name
        
        # curve definitions
        self.q = 2^448 - 2^224 - 1
        self.A = 156326
        
    def scalar_mult(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file

    def scalar_mult_vfy(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file

#H = H_SHA512()
#G = G_X25519()
#
#G.calculate_generator(H,b"PRS",b"CI",b"sid", True)

In [72]:
# 4.) Definitions of the Elligator2 test cases
#
# Elligator 2 test cases
#
#
# Testvector from the NaCl M0 testsuite from E+H
#
G = G_X25519()

etc1_in =  0x00c84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
etc1_out = 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666

etc2_in =  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
etc2_out = 0x08a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d

ourResult1 = ByteArrayToInteger(G.elligator2(etc1_in))
ourResult2 = ByteArrayToInteger(G.elligator2(etc2_in))

if (ourResult1 != etc1_out):
    print ("Elligator test case #1 failed.")
else:
    print ("Elligator test case #1 pass.")

if (ourResult2 != etc2_out):
    print ("Elligator test case #2 failed.")
else:
    print ("Elligator test case #2 pass.")
    
print ("Elligator test case #1:")
print ("In:  0x%x" % etc1_in)
print ("Out: 0x%x" % etc1_out)

print ("Elligator test case #1:")
print ("In:  0x%x" % etc2_in)
print ("Out: 0x%x" % etc2_out)


Elligator test case #1 pass.
Elligator test case #2 pass.
Elligator test case #1:
In:  0xc84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
Out: 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666
Elligator test case #1:
In:  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
Out: 0x8a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d


In [ ]:
def CPace_ISK(DSI,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = False):
    if symmetric_execution:
        concatenated_msg_transcript = oCAT(MSGa,MSGb)
        print ("  ###  Test vector for ISK calculation symmetric execution ###")
    else:
        concatenated_msg_transcript = MSGa + MSGb
        print ("  ###  Test vector for ISK calculation initiator/responder ###")
        
    string = prefix_free_cat(DSI,sid,K)+ concatenated_msg_transcript
    m = hashlib.sha512(string)
    ISK = m.digest()
    if doPrint:
        tv_output_byte_array(string, test_vector_name = "    input to final ISK hash", 
                         line_prefix = "      ", max_len = 60)
        tv_output_byte_array(ISK, test_vector_name = "    ISK result", 
                         line_prefix = "      ", max_len = 60)

    return ISK

In [81]:
def generate_test_vector(H,G):
    print ("#####  Test vector for CPace using group " + G.name + " and hash "+H.name +"  #####")

    sid = H.hash(b"sid")
    sid = sid [:16]

    PRS = b"password"
    CI = (prepend_length_to_bytes(b"Ainitiator") 
          + prepend_length_to_bytes(b"Bresponder"))

    ADa = b"ADa"
    ADb = b"ADb"

    
    g = G.calculate_generator(H,PRS,CI,sid, True)
    ya = G.sample_scalar()
    yb = G.sample_scalar()
    Ya = G.scalar_mult(ya, g)
    Yb = G.scalar_mult(yb, g)
    MSGa = prefix_free_cat(Ya,ADa)
    MSGb = prefix_free_cat(Yb,ADb)
   
    print ("  ###  Test vector for MSGa ###")
    print ("  inputs:")
    print ("    ADa =",ADa)
    tv_output_byte_array(ya, test_vector_name = "    ya", 
                         line_prefix = "      ", max_len = 60)
    tv_output_byte_array(Ya, test_vector_name = "    Ya", 
                         line_prefix = "      ", max_len = 60)
    tv_output_byte_array(MSGa, test_vector_name = "    MSGa", 
                         line_prefix = "      ", max_len = 60)
    print ("  ###  Test vector for MSGb ###")
    print ("  inputs:")
    print ("    ADb =", ADb)
    tv_output_byte_array(yb, test_vector_name = "    yb", 
                         line_prefix = "      ", max_len = 60)
    tv_output_byte_array(Yb, test_vector_name = "    Yb", 
                         line_prefix = "      ", max_len = 60)
    tv_output_byte_array(MSGb, test_vector_name = "    MSGb", 
                         line_prefix = "      ", max_len = 60)
    
    print ("  ###  Test vector for secret points K ###")
    K1 = G.scalar_mult_vfy(ya,Yb)
    K2 = G.scalar_mult_vfy(yb,Ya)
    tv_output_byte_array(K1, test_vector_name = "    scalar_mult_vfy(ya,Yb)", 
                         line_prefix = "      ", max_len = 60)
    tv_output_byte_array(K2, test_vector_name = "    scalar_mult_vfy(yb,Ya)", 
                         line_prefix = "      ", max_len = 60)

    if (K1 != K2):
        print ("Diffie-Hellman did fail!")
    K = K1
    
    ISK = CPace_ISK(G.DSI_ISK,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = False)
    ISK = CPace_ISK(G.DSI_ISK,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = True)

    print ("####  End Test vector for CPace  #####")

H = H_SHA512()
G = G_X25519()
generate_test_vector(H,G)

#    print ("\nSecret scalar ya=SHA512(\'ya\'), bytes 0...31, as integer:\n  0x%x" % ByteArrayToInteger(ya))
#    print ("Secret scalar yb=SHA512(\'yb\'), bytes 0...31, as integer:\n  0x%x" % ByteArrayToInteger(yb))

#print ("Public point Ya as integer:\n  0x%x" % ByteArrayToInteger(Ya))
#print ("Public point Yb as integer:\n  0x%x" % ByteArrayToInteger(Yb))

#print ("DH point K as integer:\n  0x%x" % ByteArrayToInteger(K1))

#print ("\nDerivation of the intermediate session key ISK:\n")
#ISK = CPace_ISK(G.DSI_ISK, sid,K,MSGa,MSGb,1)
#tv_output_byte_array(ISK, test_vector_name = "  ISK", 
#                                 line_prefix = "    ", max_len = 70)

#####  Test vector for CPace using group X25519 and hash SHA512  #####
  ####  Test vectors for calculate_generator with group X25519  ####
  Inputs
    H   = SHA512 with input block size 128 bytes.
    PRS = b'password' ; ZPAD length: 118 ; DSI = b'CPace255'
    CI = b'\nAinitiator\nBresponder'
    CI = 0a41696e69746961746f720a42726573706f6e646572
    sid = 7e4b4791d6a8ef019b936c79fb7f2c57
  Outputs
    hash generator string: (length = 32 bytes)
      5cb423cc3a5a9355bb90fceb67c97a7b5787df93faf4562789d705e3b2848d86
    after decoding to coordinate: (length = 32 bytes)
      5cb423cc3a5a9355bb90fceb67c97a7b5787df93faf4562789d705e3b2848d06
    generator g: (length = 32 bytes)
      2cddcc94b38d059a7b305bb0b8934b5b1ed45c5a5cb039f9cd00ab11ce92730d
  ###  Test vector for MSGa ###
  inputs:
    ADa = b'ADa'
    ya: (length = 32 bytes)
      bfe148a3228b32256628c91facada2563ad9e945001e855388c171
      f0f6f5142
    Ya: (length = 32 bytes)
      ebf61b5862e93a4400ea1ed6adf9b3ed8725f39d54c89cf

In [ ]:
# Test vectors for the C Code, Elligator2

print (ByteArrayToCInitializer(IntegerToByteArray(etc1_in), "EllTestCase1_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc1_out), "EllTestCase1_out"))

print (ByteArrayToCInitializer(IntegerToByteArray(etc2_in), "EllTestCase2_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc2_out), "EllTestCase2_out"))

tc3_in = bytearray(hashlib.sha512(b"LongInput").digest())
print (ByteArrayToCInitializer(tc3_in, "EllTestCase3_in"))

tc3_out = elligator2_curve25519(tc3_in)
print (ByteArrayToCInitializer(tc3_out, "EllTestCase3_out"))


In [ ]:
#

In [ ]:
#8.) Test vector output for ANSI-C, CPace

print (ByteArrayToCInitializer(G, "tc_g"))
print (ByteArrayToCInitializer(ya, "tc_ya"))
print (ByteArrayToCInitializer(Ya, "tc_Ya"))
print (ByteArrayToCInitializer(yb, "tc_yb"))
print (ByteArrayToCInitializer(Yb, "tc_Yb"))
print (ByteArrayToCInitializer(K1, "tc_K"))
print (ByteArrayToCInitializer(ISK, "tc_ISK"))

